In [28]:
import pandas as pd

from generate_cases import generate_views, view_to_natural_language
from pyetr.inference import (
    default_inference_procedure,
    classically_valid_inference_procedure,
)

In [72]:
views = generate_views(
    n_views=50,
    max_domain_size=4,
    max_conjuncts=2,
    max_disjuncts=2,
    generate_supposition=True,
    neg_prob=0.2,
    supposition_prob=0.2,
)

In [80]:
from pyetr.view import View

def views_have_nonempty_intersection(v1: View, v2: View) -> bool:
    v1_atoms = set([a if a.predicate.verifier else ~a for a in v1.atoms])
    v2_atoms = set([a if a.predicate.verifier else ~a for a in v2.atoms])
    return len(v1_atoms.intersection(v2_atoms)) > 1

def generate_premises(views: list[View]) -> list[tuple[View, ...]]:
    ret = []
    for v1 in views:
        for v2 in views:
            if all([
                views_have_nonempty_intersection(v1, v2),
                not(len(v1.stage) > 1 and len(v2.stage) > 1),
            ]):
                ret.append((v1, v2))
    return ret

In [81]:
premises = generate_premises(views)

In [82]:
len(views)

50

In [83]:
len(premises)

11

In [84]:
# Get all ordered pairs of views -- too naive
# premises = [(v1, v2) for v1 in views for v2 in views]

In [85]:
df = pd.DataFrame(premises, columns=["p1", "p2"])

In [86]:
df.head()

,p1,p2
0,{Jack()}^{~Jack()Seven()},{Jack()}^{~Jack()Seven()}
1,{Queen()}^{Ace()},{Queen()}^{Ace()}
2,{King()Nine()}^{},{King()Nine()}^{}
3,{Seven()Five()}^{},{Seven()Five()}^{}
4,{Seven()Five()}^{},{Seven()}^{Five()}


In [87]:
df["p1_nl"] = df["p1"].apply(view_to_natural_language)
df["p2_nl"] = df["p2"].apply(view_to_natural_language)

In [88]:
df.head()

,p1,p2,p1_nl,p2_nl
0,{Jack()}^{~Jack()Seven()},{Jack()}^{~Jack()Seven()},"if there is a seven and not a jack, then there...","if there is a seven and not a jack, then there..."
1,{Queen()}^{Ace()},{Queen()}^{Ace()},"if there is an ace, then there is a queen","if there is an ace, then there is a queen"
2,{King()Nine()}^{},{King()Nine()}^{},there is a king and a nine,there is a king and a nine
3,{Seven()Five()}^{},{Seven()Five()}^{},there is a seven and a five,there is a seven and a five
4,{Seven()Five()}^{},{Seven()}^{Five()},there is a seven and a five,"if there is a five, then there is a seven"


In [89]:
len(df)

11

In [90]:
from tqdm.notebook import tqdm
tqdm.pandas()

df["etr_c"] = df.progress_apply(lambda row: default_inference_procedure([row["p1"], row["p2"]]), axis=1)

  0%|          | 0/11 [00:00<?, ?it/s]

In [91]:
df.head()

,p1,p2,p1_nl,p2_nl,etr_c
0,{Jack()}^{~Jack()Seven()},{Jack()}^{~Jack()Seven()},"if there is a seven and not a jack, then there...","if there is a seven and not a jack, then there...",{0}
1,{Queen()}^{Ace()},{Queen()}^{Ace()},"if there is an ace, then there is a queen","if there is an ace, then there is a queen",{0}
2,{King()Nine()}^{},{King()Nine()}^{},there is a king and a nine,there is a king and a nine,"{0,King()Nine()}"
3,{Seven()Five()}^{},{Seven()Five()}^{},there is a seven and a five,there is a seven and a five,"{0,Seven()Five()}"
4,{Seven()Five()}^{},{Seven()}^{Five()},there is a seven and a five,"if there is a five, then there is a seven","{0,Five()}"


In [63]:
cat_c_df = df[df["etr_c"].apply(lambda x: len(x.stage) == 1 and not x.is_verum)]
cat_c_df[
    (cat_c_df["p2"].apply(lambda x: len(x.stage) > 1)) &
    (cat_c_df["p1"].apply(lambda x: len(x.stage) > 1))
]

,p1,p2,p1_nl,p2_nl,etr_c


In [64]:
cat_c_df

,p1,p2,p1_nl,p2_nl,etr_c
3580,"{Nine(),King()}^{~Jack()}","{King()}^{One(),~One(),King()}","if there is not a jack, then either there is a...","if either there is a one, or there is not a on...",{King()}
3593,"{Nine(),King()}^{~Jack()}","{One()}^{~Four(),Three(),~Three()}","if there is not a jack, then either there is a...","if either there is not a four, or there is a t...",{One()}
3601,"{Nine(),King()}^{~Jack()}","{Two()}^{Two(),~Two()}","if there is not a jack, then either there is a...","if either there is a two, or there is not a tw...",{Two()}
3614,"{Nine(),King()}^{~Jack()}","{Ten()}^{~Ten(),Ten()}","if there is not a jack, then either there is a...","if either there is not a ten, or there is a te...",{Ten()}
3656,"{Nine(),King()}^{~Jack()}","{Two()}^{Ace(),Two(),~Two()}","if there is not a jack, then either there is a...","if either there is an ace, or there is a two, ...",{Two()}
...,...,...,...,...,...
297161,"{Ten(),King()}^{~King()}","{Seven()}^{One(),~Four(),~One()}","if there is not a king, then either there is a...","if either there is a one, or there is not a fo...",{Seven()}
297170,"{Ten(),King()}^{~King()}","{One()}^{Ace(),~Ace()}","if there is not a king, then either there is a...","if either there is an ace, or there is not an ...",{One()}
297185,"{Ten(),King()}^{~King()}","{Ten()}^{~Ten(),Ten()}","if there is not a king, then either there is a...","if either there is not a ten, or there is a te...",{Ten()}
297335,"{Ten(),King()}^{~King()}","{Seven()}^{~Seven(),Seven()}","if there is not a king, then either there is a...","if either there is not a seven, or there is a ...",{Seven()}
